In [1]:
import pyspark.sql.functions as f_
from pyspark.sql.window import Window
import sys
sys.path.append('/lakehouse/default/Files/GENERALES')
import f_amr

StatementMeta(, 4d9200d0-78e8-416f-a835-f2b3b4cacd86, 3, Finished, Available, Finished)

#### Adaptación de variables respecto a BDD

In [2]:
#Curva de intentos
df_c3 = spark.read.format("delta").load("abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/272b2eb3-5441-4846-8811-625d59de802e/Tables/BDD_TLMK/curvas_p3")

#Seleccionar unicamente lo tocado en base a las fechas
df_c3 = df_c3.withColumn("TOQUE_DIA", f_.when(f_.col("FECHA_INICIO") >= f_.col("FECHA_LEAD"), 1).otherwise(0)) \
             .filter(f_.col("TOQUE_DIA") == 1)


# Ventanas
window_ciclo = Window.partitionBy("CICLO", "NO_PERSONA").orderBy("FECHA_INICIO")
window_dia = Window.partitionBy("FECHA_INICIO", "NO_PERSONA").orderBy("FECHA_INICIO")
window_semana = Window.partitionBy("NO_PERSONA", "SEMANA").orderBy("FECHA_INICIO")

# Calcular valores únicos 
df_final = df_c3.withColumns({
    "REG_UNICOS_CICLO": f_.when(f_.row_number().over(window_ciclo) == 1, 1).otherwise(0),
    "REG_UNICOS_DIA": f_.when(f_.row_number().over(window_dia) == 1, 1).otherwise(0),
    "REG_UNICOS_SEM": f_.when(f_.row_number().over(window_semana) == 1, 1).otherwise(0)
})

# Calcular métricas de llamadas
df_final = df_final.withColumns({
    "CONTESTADAS": f_.when(f_.col("DESCRIPCION") == "LLAMADA CONTESTADA", f_.col("CONTADOR")).otherwise(0),
    "EFECTIVAS": f_.when(
        (~f_.col("TIPIFICACION").contains("NO CONTESTA")) &
        (~f_.col("TIPIFICACION").contains("CORT")) &
        (f_.col("TIPIFICACION").contains("CONTESTADORA")),
        f_.col("CONTESTADAS")
    ).otherwise(0),
    "NO_EFECTIVAS": f_.when(
        (f_.col("TIPIFICACION").contains("NO CONTESTA")) |
        (f_.col("TIPIFICACION").contains("CORT")) |
        (f_.col("TIPIFICACION").contains("CONTESTADORA")),
        f_.col("CONTESTADAS")
    ).otherwise(0),
    "CORTADAS": f_.when(
        (f_.col("TIPIFICACION").contains("NO CONTESTA")) |
        (f_.col("TIPIFICACION").contains("CORT")),
        f_.col("CONTESTADAS")
    ).otherwise(0)
}).drop('RSP_ESCUELA_OK','MEIDUM','ESTADO','BANNER')

# Adición de variables
df_sitio = spark.read.format("delta").load("abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/272b2eb3-5441-4846-8811-625d59de802e/Tables/BDD_TLMK/AGR_SITIO")
df_sitio = df_sitio.withColumnRenamed('CICLO_BI', 'CICLO2')
df_pros = spark.read.format("delta").load("abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/272b2eb3-5441-4846-8811-625d59de802e/Tables/BDD_TLMK/REGISTROS_PROSPECCION")

# Prospección
df_join = (df_final.join(df_pros, (df_final["NO_PERSONA"] == df_pros["no_de_persona"]), "left")).drop('AGRUPADOR_SITIO','BANNER','MEDIUM','ESTADO')
df_join = df_join.dropDuplicates(["RECORDID"])

# Digital
df_join2 = (df_join.join(df_sitio, (df_join["NO_PERSONA"] == df_sitio["no_de_persona"]) & (df_join["CICLO"] == df_sitio["CICLO2"]), "left")).drop('NO_DE_PERSONA','CICLO2')
df_join2 = df_join2.dropDuplicates(["RECORDID"])

# Agrupación final
columnas_agrupacion = [
    "AGRUPADOR_SITIO",
    "AGRUPADOR_ROBOT", 
    "AGRUPADOR_TIPIFICACION",
    "BANNER",
    "BLN",
    "CAMPUS_BI",
    "CICLO",
    "DESCRIPCION",
    "ESTATUS_CAMPUS",
    "FECHA_INICIO",
    "FECHA_LEAD",
    "FUENTE", 
    "FUENTE_AGRUPADA",
    "HOMOLOGADO",
    "LINEA_AGRUPADA",
    "LINEA_NEGOCIO_BI",
    "MERCADO",
    "MODALIDAD",
    "MODALIDAD_AGRUPADA",
    'NO_PERSONA',
    "OPTIMO_MARCABLE", 
    "PROGRAMA_BI", 
    "TEAM_PRODUCTOR",
    "TIPIFICACION",
    "TIPO_BASE_BI", 
    "TIPO_INGRESO_BI",
    "TIPO_TRAMITE",
    "TIPO_RECICLADO", 
    #"ULTIMO_RESULTADO_AGENTE_CODIGO",
    #"ULTIMO_RESULTADO_AGENTE_TEXTO",
    #"ULTIMO_RESULTADO_MARCADOR_CODIGO",
    "VARIABLE",
    "UTILIDAD",
    "VERTICAL_BI"       
]

df_agrupado = df_join2.groupBy(columnas_agrupacion).agg(
    f_.sum("REG_UNICOS_CICLO").alias("REG_UNICOS_CICLO"),
    f_.sum("REG_UNICOS_DIA").alias("REG_UNICOS_DIA"),
    f_.sum("REG_UNICOS_SEM").alias("REG_UNICOS_SEM"),
    f_.sum("CONTADOR").alias("INTENTOS")
    #f_.sum("CONTESTADAS").alias("CONTESTADAS"),
    #f_.sum("CORTADAS").alias("CORTADAS"),
    #f_.sum("EFECTIVAS").alias("EFECTIVAS"),
    #f_.sum("NO_EFECTIVAS").alias("NO_EFECTIVAS")
    )

StatementMeta(, 4d9200d0-78e8-416f-a835-f2b3b4cacd86, 4, Finished, Available, Finished)

#### Normalización básica proyecto score

###### Selección de columnas iniciales

In [3]:
#Fuente
curva_bdd_uvm = df_agrupado

# Limpieza de columnas
seleccion_1 = \
{ 'BLN':'BLN',
 'CAMPUS_BI':'CAMPUS_BI',
 'CICLO':'CICLO_BI',
 'DESCRIPCION':'DESCRIPCION',
 'FECHA_INICIO':'FECHA',
 'FECHA_LEAD':'FECHA_LEAD',
 'FUENTE':'FUENTE',
 'FUENTE_AGRUPADA':'FUENTE_AGRUPADA',
 'INTENTOS':'INTENTOS', 
 'LINEA_AGRUPADA':'LINEA_AGRUPADA',
 'LINEA_NEGOCIO_BI':'LINEA_NEGOCIO_BI',
 'MERCADO':'MERCADO',
 'MODALIDAD_AGRUPADA':'MODALIDAD_AGRUPADA', 
 'MODALIDAD':'MODALIDAD_BI',  
 'NO_PERSONA':'NO_DE_PERSONA',
 'PROGRAMA_BI':'PROGRAMA_BI',
 'REG_UNICOS_CICLO':'REG_UNICOS_CICLO',
 'REG_UNICOS_DIA':'REG_UNICOS_DIA',
 'REG_UNICOS_SEM':'REG_UNICOS_SEM',
 'TEAM_PRODUCTOR':'TEAM_PRODUCTOR',
 'TIPO_INGRESO_BI':'TIPO_DE_INGRESO_BI',
 'UTILIDAD':'UTILIDAD',
 'VARIABLE':'VARIABLE',
 'VERTICAL_BI':'VERTICAL_BI'
}

curva_bdd_uvm = f_amr.select_cols(curva_bdd_uvm, seleccion_1)
curva_bdd_uvm = curva_bdd_uvm.withColumn('AÑO',f_.lit(2025))
upper = ['BLN', 'CAMPUS_BI', 'DESCRIPCION', 'FUENTE', 'FUENTE_AGRUPADA', 'LINEA_AGRUPADA', 'LINEA_NEGOCIO_BI', 'MERCADO', 'MODALIDAD_AGRUPADA', 'MODALIDAD_BI', 'PROGRAMA_BI', 'TEAM_PRODUCTOR', 'TIPO_DE_INGRESO_BI', 'VERTICAL_BI']
for i in upper:
    curva_bdd_uvm = curva_bdd_uvm.withColumn(i, f_.upper(f_.col(i)))
curva_bdd_uvm = curva_bdd_uvm.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'FECHA')))


StatementMeta(, 4d9200d0-78e8-416f-a835-f2b3b4cacd86, 5, Finished, Available, Finished)

###### Adición y corrección de columnas básicas KPI

- Catálogos

In [4]:
#Fuente
funnel_uvm_a = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/5182b19a-e57c-4163-83a9-b0b90997f691/Files/ANALYTIKUS/FUNNEL_UVM_A.parquet')
funnel_uvm_a = funnel_uvm_a.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
#Limpieza de columnas
seleccion_2 = \
{'CAMPUS_BI':'CAMPUS_BI',
'REGION_BI':'REGION_BI',
'TIPO_RECICLADO':'TIPO_RECICLADO',
'TRAMITE':'TRAMITE',
'CICLO_BI':'CICLO_BI',
'ESTATUS_CAMPUS':'ESTATUS_CAMPUS',
'ETAPA':'ETAPA',
'FECHA':'FECHA',
'NO_DE_PERSONA':'NO_DE_PERSONA',
'RECICLADO':'RECICLADO',
'TIPO_TRAMITE':'TIPO_TRAMITE',
'ZONA_BI':'ZONA_BI'}

funnel_uvm_a = f_amr.select_cols(funnel_uvm_a, seleccion_2)
funnel_uvm_a = f_amr.uniques(funnel_uvm_a, ['NO_DE_PERSONA', 'CICLO_BI', 'ETAPA'], [('NO_DE_PERSONA', 'asc'),('FECHA', 'desc')])
funnel_uvm_a = funnel_uvm_a.filter(f_.col('ETAPA') == 'LEADS').drop('ETAPA', 'FECHA')
cat_tramite = funnel_uvm_a.select('NO_DE_PERSONA', 'CICLO_BI', 'RECICLADO', 'TIPO_TRAMITE', 'TRAMITE').distinct()
cat_campus = funnel_uvm_a.select('CAMPUS_BI', 'REGION_BI', 'ESTATUS_CAMPUS', 'ZONA_BI').distinct()

StatementMeta(, 4d9200d0-78e8-416f-a835-f2b3b4cacd86, 6, Finished, Available, Finished)

- Ajuste columnas KPI

In [5]:
# Adición selectiva
# Tramite
match_tr_1 = curva_bdd_uvm.join(cat_tramite,['NO_DE_PERSONA','CICLO_BI'], 'inner').dropDuplicates(['INDICE'])
no_match_tr_left_1 = curva_bdd_uvm.join(cat_tramite,['NO_DE_PERSONA','CICLO_BI'], 'left_anti')
no_match_tr_right_1 = cat_tramite.join(curva_bdd_uvm,['NO_DE_PERSONA','CICLO_BI'], 'left_anti')

match_tr_2 = no_match_tr_left_1.join(no_match_tr_right_1.drop('CICLO_BI'), 'NO_DE_PERSONA', 'left').dropDuplicates(['INDICE'])
curva_bdd_uvm = match_tr_1.unionByName(match_tr_2, allowMissingColumns=True)

# Campus
curva_bdd_uvm = curva_bdd_uvm.join(cat_campus,'CAMPUS_BI', 'left').dropDuplicates(['INDICE'])

# Normalización post uniones
curva_bdd_uvm = curva_bdd_uvm.withColumn('CICLO_LAUREATE',\
f_.when(f_.col("CICLO_BI").endswith('1'), 'C1')\
.when(f_.col("CICLO_BI").endswith('2'), 'C2')\
.when(f_.col("CICLO_BI").endswith('3'), 'C3').otherwise(None))

curva_bdd_uvm = curva_bdd_uvm.withColumn("CICLO", f_.concat_ws("-", f_.substring(f_.col("CICLO_BI"), 3, 2),  f_.col("CICLO_BI").endswith('1')))

# Adición de Semana
calendario = spark.read.option('header', True).option('inferSchema', True).csv('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/5182b19a-e57c-4163-83a9-b0b90997f691/Files/ANALYTIKUS/CALENDARIO_2025.csv')
calendario = calendario.withColumn('FECHA', f_.to_date('FECHA', 'dd/MM/yyyy'))
calendario = f_amr.cast_cols(calendario, {'SEMANA':'int'})

curva_bdd_uvm = curva_bdd_uvm.join(calendario.select('FECHA', 'SEMANA'), 'FECHA', 'left').drop('INDICE')

StatementMeta(, 4d9200d0-78e8-416f-a835-f2b3b4cacd86, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import current_timestamp
curva_bdd_uvm = curva_bdd_uvm.withColumn("ACTUALIZACION", f_.expr("current_timestamp() - INTERVAL 6 HOURS"))
p1= 'abfss://8c61db5f-46e1-4c72-8d6f-648bf05cacac@onelake.dfs.fabric.microsoft.com/0d2e9829-8a43-4592-a9d7-db37f7032a1c/Files/BASES_EN_PROCESO/CURVA_INTENTOS_UVM'
curva_bdd_uvm.repartition(10).write.mode("overwrite").parquet(p1)

StatementMeta(, 4d9200d0-78e8-416f-a835-f2b3b4cacd86, 8, Finished, Available, Finished)